In [8]:
#Imports 
import sys
import os

sys.path.append(os.getenv("CODE_PATH"))
sys.path.append(os.getenv("FIN_DATABASE_PATH"))

import plotly.graph_objects as go
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from Pre_Processing.pre_processing import PreProcessing
from data_fetcher import DataFetcher
from pipeline import Pipeline
from fs_pre_processing import PreProcessingFinancials
from metrics import CalculateMetrics
import openai
import streamlit as st
import datetime

In [4]:
pipe = Pipeline(tickers)
data = pipe.pipeline(combine = True)

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
GENERAL_COLUMNS = ['company_name', 'start_date', 'end_date', 'filing_date',
       'fiscal_period', 'fiscal_year', 'acceptance_datetime',
       'timeframe', 'tickers', 'sic']
SECTIONS = ['balance_sheet', 'income_statement', 'cash_flow_statement', 'comprehensive_income']


In [1]:
#Selecting some tickers to analyse
# tickers = ['AAPL', 'MSFT']
tickers = ['MSFT', 'AAPL']

In [ ]:
#Fetching data from our SQL database

fetch_data = DataFetcher(tickers)
company_data = fetch_data.get_company_data()

#Pre Processing the data
prepocess = PreProcessingFinancials(company_data, SECTIONS, tickers)
data_dict, df = prepocess.preprocess_financials(GENERAL_COLUMNS)

#Calculating Financial ratios
metrics = CalculateMetrics(df)
final_data = metrics.calculate_metrics()

In [ ]:
#List with values we will be using in the dashboard dropdowns

tickers = final_data.index.get_level_values(0).unique()
timeframes = final_data['timeframe'].unique()
metrics = ['gross_margin',
           'operating_margin',
           'net_profit_margin',
           'ROA',
           'ROE',
           'current_ratio',
           'quick_ratio',
           'debt_to_equity',
           'interest_coverage',
           'R&D_ratio']
graph_types = ['Line', 'Bar', 'Scatter']



In [ ]:
# import openai
# import os
# import pandas as pd

# openai.api_key = os.getenv("OPENAI_KEY")

# # filtering for a specific company
# selected_ticker = 'AAPL'  
# selected_company_data = final_data.xs(selected_ticker, level='ticker')  

# # Extracting financial metrics
# current_ratio = selected_company_data['current_ratio']
# debt_to_equity = selected_company_data['debt_to_equity']
# net_income = selected_company_data[('income_statement', 'net_income_loss')]
# operating_margin = selected_company_data['operating_margin']

# # Extracting the industry using SIC code
# sic_code = selected_company_data['sic'].iloc[0]
# industry = map_sic_to_industry(sic_code)  


prompt = (f"Generate a concise financial analysis for a company in the {industry} industry with the following metrics: "
          f"gross margin of {gross_margin}%, operating margin of {operating_margin}%, net profit margin of {net_profit_margin}%, "
          f"return on equity (ROE) of {ROE}%, current ratio of {current_ratio}, debt-to-equity ratio of {debt_to_equity}, "
          f"and R&D ratio of {R_and_D_ratio}%. Compare these figures to typical industry benchmarks and provide a straightforward "
          f"assessment of the company's financial health. Keep the commentary under 3-4 sentences without unnecessary explanations or filler content.")



# response = openai.chat.completions.create(
#     model="gpt-3.5-turbo",  
#     messages=[
#         {"role": "system", "content": "You are a financial analyst."},  
#         {"role": "user", "content": prompt}
#     ],
#     max_tokens=150
# )

# # Extract the generated commentary
# commentary = response.choices[0].message.content


# # commentary
# print(commentary)


In [ ]:
sic_mapping = pd.read_csv("sic_mapping.csv")
sic_mapping

In [ ]:
office_to_industry = {
    'Industrial Applications and Services': 'Manufacturing',
    'Office of Energy & Transportation': 'Energy & Transportation',
    'Office of Real Estate & Construction': 'Real Estate & Construction',
    'Office of Manufacturing': 'Manufacturing',
    'Office of Life Sciences': 'Healthcare & Life Sciences',
    'Office of Technology': 'Technology & Innovation',
    'Office of Trade & Services': 'Trade & Services',
    'Office of Finance': 'Finance',
    'Office of Structured Finance': 'Finance',
    'Office of Finance or Office of Crypto Assets': 'Finance',
    'Office of Crypto Assets': 'Finance',
    'Office of International Corp Fin': 'Finance'
}

# Map Offices to Consolidated Industries in the df
sic_mapping['Consolidated Industry'] = sic_mapping['Office'].map(office_to_industry)

#Function to Map SIC Code to Industry
def map_sic_to_consolidated_industry(sic_code, df):
    # Retrieve the row matching the SIC Code
    industry = df.loc[df['SIC Code'] == sic_code, 'Consolidated Industry']
    # Return the industry if found, else 'Unknown Industry'
    return industry.values[0] if not industry.empty else 'Unknown Industry'

In [ ]:
map_sic_to_consolidated_industry(2070, sic_mapping)

In [ ]:
sic_mapping['Industry_Group'] = sic_mapping['Office'].apply(lambda x: x.split()[-1] if pd.notnull(x) else None)

In [ ]:
sic_to_industry = sic_mapping.groupby('Industry_Group')['SIC Code'].apply(list).to_dict()

In [ ]:
def map_sic_to_industry(sic_code, sic_to_industry=sic_to_industry):
    for industry, sic_codes in sic_to_industry.items():
        if sic_code in sic_codes:
            return industry
    return None


In [ ]:
sic_mapping['Office'].unique()

In [ ]:
# Caching to optimise data fetching
@st.cache_data
def get_cached_data(tickers, timespan='daily'):
    if isinstance(tickers, str):
        tickers = [tickers]
    tickers = tuple(sorted(tickers))
    # pipeline process or data fetching here
    pipe = Pipeline(list(tickers))
    data = pipe.pipeline(combine=True, timespan=timespan)
    return data

def fetch_and_process_data(tickers):
    #Fetching data from the SQL database
    fetch_data = DataFetcher(tickers)
    company_data = fetch_data.get_company_data()
    
    # Preprocess the data
    preprocess = PreProcessingFinancials(company_data, SECTIONS, tickers)
    data_dict, df = preprocess.preprocess_financials(GENERAL_COLUMNS)
    
    # Calculate financial ratios
    metrics = CalculateMetrics(df)
    final_data = metrics.calculate_metrics()

    return final_data


st.title("Financial Dashboard")

# Select ticker(s)
selected_tickers = st.multiselect(
    "Select Ticker(s)", 
    options=tickers, 
    default=[tickers[0]]  # Default to the first ticker
)

# Select timespan
selected_timespan = st.selectbox(
    "Select Timespan", 
    options=['minute', 'hour', 'daily'], 
    index=2  # Default to 'daily'
)


if selected_tickers:
    final_data = fetch_and_process_data(selected_tickers)


col1, col2 = st.columns(2)

# Column 1: Financial Ratios Graph
with col1:
    st.subheader("Financial Ratios")

    # Select timeframe, metric, and graph type
    selected_timeframe = st.selectbox("Select Timeframe", timeframes, index=0)
    selected_metric = st.selectbox("Select Metric", metrics, index=0)
    selected_graph_type = st.selectbox("Select Graph Type", graph_types, index=0)

    # Plotting
    if selected_tickers:
        fig = go.Figure()
        
        for ticker in selected_tickers:
            # Using `.xs()` to filter based on multi-index level 'ticker'
            filtered_df = final_data.xs(ticker, level='ticker')  

            if selected_timeframe == 'quarterly':
                df_ticker = filtered_df[filtered_df['timeframe'] == 'quarterly']
                df_ticker.sort_values(by='filing_date', ascending=True, inplace=True)
                x_axis = df_ticker['period'].astype(str)
            else:
                df_ticker = filtered_df[filtered_df['timeframe'] == selected_timeframe]
                x_axis = df_ticker['fiscal_year']

            # Generate graph based on selected graph type
            if selected_graph_type == 'Line':
                fig.add_trace(go.Scatter(
                    x=x_axis,
                    y=df_ticker[selected_metric],
                    mode='lines+markers',
                    name=ticker
                ))
            elif selected_graph_type == 'Bar':
                fig.add_trace(go.Bar(
                    x=x_axis,
                    y=df_ticker[selected_metric],
                    name=ticker
                ))
            elif selected_graph_type == 'Scatter':
                fig.add_trace(go.Scatter(
                    x=x_axis,
                    y=df_ticker[selected_metric],
                    mode='markers',
                    name=ticker
                ))

        # Update layout and plot the figure
        fig.update_layout(
            title=f"{selected_metric} for {', '.join(selected_tickers)} ({selected_timeframe})",
            xaxis_title='Period',
            yaxis_title=selected_metric,
            legend_title="Ticker"
        )
        st.plotly_chart(fig)

# Column 2: Stock Data Graph
with col2:
    st.subheader("Stock Data")

    # Fetch the stock data using the pipeline
    if selected_tickers:
        stock_data = get_cached_data(selected_tickers, selected_timespan)  # Fetching stock data dynamically using the function we previously created

        # Update technical indicators options dynamically
        non_indicator_columns = ['id', 'date', 'timestamp', 'ticker']
        indicator_columns = set(stock_data.columns) - set(non_indicator_columns)
        indicator_options = [col for col in indicator_columns]

        # Select technical indicators
        selected_indicators = st.multiselect(
            "Select Technical Indicator(s)", 
            options=indicator_options, 
            default=indicator_options[:1]  # Default to first indicator
        )

        fig = go.Figure()

        for ticker in selected_tickers:
            ticker_data = stock_data[stock_data['ticker'] == ticker]  # Fetching stock data for each ticker

            for indicator in selected_indicators:
                fig.add_trace(go.Scatter(
                    x=ticker_data['date'],  # X-axis is 'date' column
                    y=ticker_data[indicator],  # Y-axis is the selected technical indicator
                    mode='lines',
                    name=f"{ticker} - {indicator}"
                ))

       
        fig.update_layout(
            title=f"{', '.join(selected_indicators)} for {', '.join(selected_tickers)} ({selected_timespan})",
            xaxis_title='Date',
            yaxis_title='Value',
            legend_title="Ticker - Indicator"
        )
        

        st.plotly_chart(fig)


